In [13]:
import pandas as pd
from datetime import datetime
from datetime import timedelta
import numpy as np
from scipy.stats.stats import pearsonr
import matplotlib.pyplot as plt
import json, os, collections

In [14]:
playtest_files = ["2018-11-14_Playtest/anonymized_playtest5_data.csv", "2019-01-07_Playtest/anonymized_playtest6_data.csv", "2019-01-31_Playtest/anonymized_playtest7_data.csv"]

playtests = []

for playtest in playtest_files:
    playtests.append(pd.read_csv(playtest, sep=";"))
    
# Set the playtest index we want to focus on
playtest = 1


In [15]:
def get_shapes(puzzle):
    shapes = []
    if puzzle is None:
        return shapes
    for shape_data in puzzle.get("shapeData"):
        shapes.append(shape_data.get('shapeType'))
    return shapes    

In [16]:
puzzles = collections.OrderedDict()

with open(f"{os.path.dirname(playtest_files[playtest])}/StreamingAssets/config.json") as f:
    asset_config = json.load(f)

for puzzle_sets in asset_config.get("puzzleSets"):
    for puzzle_file in puzzle_sets.get("puzzles"):
        with open(f"{os.path.dirname(playtest_files[playtest])}/StreamingAssets/{puzzle_file}.json") as f:
            puzzle_details = json.load(f)
            puzzles[puzzle_details.get("puzzleName")] = puzzle_details

In [17]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', -1)
#all_playtest_data = pd.concat([playtest5_data, playtest6_data, playtest7_data])
all_playtest_data = pd.concat([playtests[playtest]])
all_playtest_data = all_playtest_data.combine_first(pd.io.json.json_normalize(all_playtest_data.data.apply(json.loads)))

In [18]:
# puzzleDict[user~puzzle_id] = {}
puzzleDict = {}

# For each session
for user in all_playtest_data['session_id'].unique():
    # Get the sessions
    user_events = all_playtest_data[all_playtest_data['session_id'] == user]
    activePuzzle = None
    for enum, event in user_events.iterrows():
        #print(('{} - {}').format(event['time'], event['type']))
        
        if(event['type'] == 'ws-start_level'):
            activePuzzle = event['task_id']
            print (activePuzzle)

        # If they are not playing a puzzle we do not do anything and continue
        if(activePuzzle is None):
            continue

    # Analyze when puzzle is finished or user left
        # Measure time, attempts, completion and actions
        if(event['type'] in ['ws-puzzle_complete', 'ws-exit_to_menu', 'ws-disconnect']):
            #print('\\finish\\')
            activePuzzle = None
        else:
            # Set the active puzzle to be this one
            all_playtest_data.at[enum, 'task_id'] = activePuzzle

Design a Puzzle
Pi Henge
One Cube
Separated Cubes
Rotate a Pyramid
Match Silhouettes
One Cube
One Cube
Separated Cubes
Rotate a Pyramid
Match Silhouettes
Removing Objects
Stretch a Ramp
Max 2 Cubes
Stack 2 Ramps
Scaling Round Objects
Square Cross-Sections
Pi Henge
Bird Fez
45-Degree Rotations
Square Cross-Sections
Pyramids are Strange
Cubes Obscure Spheres
One Cube
One Cube
Stretch a Ramp
Stack 2 Ramps
Scaling Round Objects
Square Cross-Sections
Bird Fez
Pi Henge
45-Degree Rotations
Pyramids are Strange
Cubes Obscure Spheres
Shape Limits
Not Bird
Angled Silhouette
Square Cross-Sections
Angled Silhouette
Design a Puzzle
Design a Puzzle
Separated Cubes
Separated Cubes
Rotate a Pyramid
Match Silhouettes
Max 2 Cubes
Removing Objects
Design a Puzzle
Design a Puzzle
Design a Puzzle
Design a Puzzle
Sandbox
Sandbox
Not Bird
Not Bird
One Cube
Not Bird
Separated Cubes
Rotate a Pyramid
Match Silhouettes
Design a Puzzle
Not Bird
Sandbox
Not Bird
Angled Silhouette
Shape Limits
Shape Limits
Cubes Ob

In [19]:
all_playtest_data['time'] = pd.to_datetime(all_playtest_data['time'])



In [20]:
all_attempts = all_playtest_data.task_id.value_counts()


In [21]:
complete_puzzles = all_playtest_data.loc[all_playtest_data['type'] == 'ws-puzzle_complete']
successful_attempts = complete_puzzles.task_id.value_counts()

In [22]:
created_shapes = all_playtest_data.loc[all_playtest_data['type'] == 'ws-create_shape']
shapes_used = collections.defaultdict(list)
for shapes, value in created_shapes.iterrows():
    #print (f"{value.get('task_id')} {int(value.get('shapeType'))}")
    shapes_used[value.get('task_id')].append(int(value.get('shapeType')))
    #print (shapes['task_id'])

In [23]:
for attempt, value in all_attempts.items():
    success_value = successful_attempts.get(attempt, 0)
    shapes = get_shapes(puzzles.get(attempt))
    print (f"Puzzle {attempt: <25} Attempts {value: < 5} / Success {success_value: < 5} : {round(((success_value/value) * 100), 1): < 5}% Shapes: {shapes}")
    times_used = collections.Counter(shapes_used[attempt])
    #print (f"Shapes Tried: {shapes_used[attempt]}")
    print (f"Shapes Used Total: {times_used}")

Puzzle Design a Puzzle           Attempts  1252 / Success  0    :  0.0 % Shapes: []
Shapes Used Total: Counter({1: 33, 4: 23, 5: 18, 2: 11, 3: 6, 6: 5})
Puzzle Pyramids are Strange      Attempts  1050 / Success  6    :  0.6 % Shapes: [2, 2, 2, 2]
Shapes Used Total: Counter({2: 48})
Puzzle Scaling Round Objects     Attempts  767  / Success  7    :  0.9 % Shapes: [4, 2]
Shapes Used Total: Counter({4: 13, 5: 13, 6: 1, 1: 1})
Puzzle Cubes Obscure Spheres     Attempts  724  / Success  6    :  0.8 % Shapes: [6, 4, 4, 1]
Shapes Used Total: Counter({1: 33, 4: 27, 6: 11})
Puzzle Not Bird                  Attempts  647  / Success  3    :  0.5 % Shapes: [6, 4, 2]
Shapes Used Total: Counter({1: 10, 4: 9, 6: 6, 5: 6, 2: 5, 3: 2})
Puzzle Bird Fez                  Attempts  635  / Success  8    :  1.3 % Shapes: [6, 5, 4]
Shapes Used Total: Counter({1: 9, 6: 8, 5: 5, 4: 3, 2: 3})
Puzzle Rotate a Pyramid          Attempts  629  / Success  10   :  1.6 % Shapes: [2]
Shapes Used Total: Counter({2: 20})
Pu

In [24]:

all_playtest_data.type.value_counts()

ws-mode_change                1520
ws-rotate_shape               1202
ws-rotate_view                1088
ws-scale_shape                1004
ws-move_shape                 958 
ws-select_shape               927 
ws-deselect_shape             832 
ws-create_shape               649 
ws-puzzle_started             267 
ws-start_level                245 
ws-check_solution             243 
ws-click_nothing              237 
ws-exit_to_menu               200 
ws-snapshot                   194 
ws-delete_shape               158 
ws-puzzle_complete            143 
ws-undo_action                56  
ws-toggle_snapshot_display    50  
ws-login_user                 22  
ws-restart_puzzle             22  
ws-start_game                 22  
ws-select_shape_add           20  
ws-paint                      14  
ws-toggle_paint_display       12  
ws-palette_change             12  
ws-redo_action                3   
start_game                    1   
Name: type, dtype: int64